# SQL Server 2019 Data Virtualization - Using Polybase to query Sharepoint
This notebook contains an example of how to use external tables to query data in Sharepoint without moving data. You may need to change identity, secret, connection, database, schema, and remote table names to work with your Sharepoint Server. This example uses Sharepoint Server 2019.

This notebook also assumes you are using SQL Server 2019 Release Candidate or later on Windows, have the Polybase feature has been installed and enabled, and have installed the Sharepiont ODBC Driver from CDATA from http://cdn.cdata.com/help/RSE/odbc/.

This notebook uses the sample WideWorldImporters sample database but can be used with any user database.

## Step 0: Deploy Sharepoint 2019 Server and create a site

Create a Sharepoint 2019 Server and create a site that can be used as an EXTERNAL DATA SOURCE.

For purposes of this demo create a Sharepoint List which you can substitute the name in the EXTERNAL TABLE LOCATION.



## Step 1: Create a master key
Create a master key to encrypt the database credential

In [14]:
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

## Step 2: Create a database credential
The database credential contains the IDENTITY (login) and SECRET (password) of an authorized user on the Sharepoint Server.

In [15]:
CREATE DATABASE SCOPED CREDENTIAL SharepointCredentials   
WITH IDENTITY = '<sharepoint user>', Secret = '<sharepoint user password>'
GO

Commands completed successfully.

Total execution time: 00:00:00.005

## Step 3: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

The LOCATION syntax is <datasourcetype>:<connection string>.

datasourcetype can be sqlserver, oracle, teradata, mongodb, or odbc (Windows only)
The connection string depends on the datasourcetype

Since access to Sharepoint is through an ODBC driver not installed by default, the LOCATION is the name of the host for the Sharepoint Server (or IP Address). Use the CONNECTION_OPTIONS for the ODBC data source connection string including the name of the driver, the name of the host (or IP address), and the Sharepiont Site URL.

In [16]:
CREATE EXTERNAL DATA SOURCE SharepointServer
WITH ( 
LOCATION = 'odbc://<sharepoint server host ip>',
CONNECTION_OPTIONS = 'DRIVER={CData ODBC Driver for SharePoint};ServerNode = <sharepoint server host ip>;URL=http://<sharepoint site URL>',
PUSHDOWN = ON,
CREDENTIAL = SharepointCredentials
)
GO

Commands completed successfully.

Total execution time: 00:00:00.121

## Step 4: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [17]:
CREATE SCHEMA sharepoint
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 5: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. The name of the tables for the external table can be your choice. But the columns must be specified in the same order with the same name as they are defined in the remote table. Furthermore, local data types must be compatible with the remote table.

The WITH clause specifies a LOCATION. This LOCATION is different than the EXTERNAL DATA SOURCE. For Sharepoint using the CDATA driver, the LOCATION is the name of an object you create on the sharepoint site. In this example, you will use a Sharepoint list name. The column names match the field properties of the Sharepoint list name.

In [18]:
CREATE EXTERNAL TABLE sharepoint.keynotelist
(
[ID] INT NOT NULL, 
[Version] NVARCHAR(510) COLLATE Latin1_General_100_CI_AS, 
[App Created By] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[App Created By_ID] INT, 
[App Modified By] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[App Modified By_ID] INT, 
[Attachments] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Created By] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Created By_ID] INT, 
[Compliance Asset Id] NVARCHAR(510) COLLATE Latin1_General_100_CI_AS, 
[Content Type] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Created] DATETIME2(3), 
[Type] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Edit] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Modified By] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Modified By_ID] INT, 
[Folder Child Count] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Folder Child Count_ID] INT, 
[Item Child Count] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS, 
[Item Child Count_ID] INT, 
[Modified] DATETIME2(3),
[Owner] NVARCHAR(4000) COLLATE Latin1_General_100_CI_AS,
[Title] NVARCHAR(510) COLLATE Latin1_General_100_CI_AS
)
WITH (
LOCATION='keynotelist',
DATA_SOURCE=SharepointServer
)
GO

Commands completed successfully.

Total execution time: 00:00:00.296

## Step 6: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [19]:
CREATE STATISTICS keynoteliststats ON sharepoint.keynotelist ([Owner]) WITH FULLSCAN
GO

Commands completed successfully.

Total execution time: 00:00:00.413

## Step 7: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [20]:
SELECT Title, Owner FROM sharepoint.keynotelist
GO

(6 rows affected)

Total execution time: 00:00:00.405

Title,Owner
Prep keynote demo for PASS,Bob Ward
Order Texas BBQ,Conor Cunningham
Build demo for PASS keynote,Bob Ward
Get Dallas Stars Jerseys,Conor Cunningham
Rehearse final keynote demo for PASS,Bob Ward
Steal Rohan's Seahawks Jersey,Conor Cunningham


## Step 8: Filter on a set of rows

In [21]:
SELECT Title, Owner FROM sharepoint.keynotelist
WHERE Owner = 'Bob Ward'
GO

(3 rows affected)

Total execution time: 00:00:00.401

Title,Owner
Prep keynote demo for PASS,Bob Ward
Build demo for PASS keynote,Bob Ward
Rehearse final keynote demo for PASS,Bob Ward
